Pre-Task - Setting Up Ollama Server

In [ ]:
!pip install -q transformers torch datasets accelerate tqdm bitsandbytes huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 18.8 MB/s eta 0:00:00


In [ ]:
!apt-get update -qq

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
!apt-get install -y zstd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 66 not upgraded.


In [ ]:
# verbose attempt showing headers and HTTP status
!curl -v -fsSL https://ollama.com/install.sh -o /tmp/ollama_install.sh || echo "curl failed with exit $?"

# show saved response (if any)
!ls -l /tmp/ollama_install.sh && sed -n '1,200p' /tmp/ollama_install.sh

*   Trying 34.36.133.15:443...
* Connected to ollama.com (34.36.133.15) port 443 (#0)
* ALPN, offering h2
* ALPN, offering http/1.1
*  CAfile: /etc/ssl/certs/ca-certificates.crt
*  CApath: /etc/ssl/certs
* TLSv1.0 (OUT), TLS header, Certificate Status (22):
} [5 bytes data]
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
} [512 bytes data]
* TLSv1.2 (IN), TLS header, Certificate Status (22):
{ [5 bytes data]
* TLSv1.3 (IN), TLS handshake, Server hello (2):
{ [122 bytes data]
* TLSv1.2 (IN), TLS header, Finished (20):
{ [5 bytes data]
* TLSv1.2 (IN), TLS header, Supplemental data (23):
{ [5 bytes data]
* TLSv1.3 (IN), TLS handshake, Encrypted Extensions (8):
{ [19 bytes data]
* TLSv1.3 (IN), TLS handshake, Certificate (11):
{ [4053 bytes data]
* TLSv1.3 (IN), TLS handshake, CERT verify (15):
{ [264 bytes data]
* TLSv1.3 (IN), TLS handshake, Finished (20):
{ [52 bytes data]
* TLSv1.2 (OUT), TLS header, Finished (20):
} [5 bytes data]
* TLSv1.3 (OUT), TLS change cipher, Change cipher sp

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
import time

ollama_process = subprocess.Popen(['ollama', 'serve'],
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)

print("Starting Ollama server...")
print("✓ Ollama server started")

Starting Ollama server...
✓ Ollama server started


In [ ]:
!ollama pull llama3.2:1b

In [ ]:
from huggingface_hub import login
login()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Task 1

Without Ollama Server - Sequential Execution

In [ ]:
%%writefile llama_mmlu_astronomy.py
"""
Llama 3.2-1B MMLU Evaluation Script - ASTRONOMY ONLY
"""

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import json
from tqdm.auto import tqdm
from datetime import datetime
import sys

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
USE_GPU = True
MAX_NEW_TOKENS = 1
QUANTIZATION_BITS = 4  # Using 4-bit for faster loading in Colab
MMLU_SUBJECTS = ["astronomy"]  # ONLY ASTRONOMY

def get_quantization_config():
    if QUANTIZATION_BITS == 4:
        return BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )
    return None

def load_model_and_tokenizer():
    print(f"Loading {MODEL_NAME} for ASTRONOMY evaluation...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    quant_config = get_quantization_config()

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=quant_config,
        device_map="auto",
        low_cpu_mem_usage=True
    )
    model.eval()
    return model, tokenizer

def format_mmlu_prompt(question, choices):
    choice_labels = ["A", "B", "C", "D"]
    prompt = f"{question}\n\n"
    for label, choice in zip(choice_labels, choices):
        prompt += f"{label}. {choice}\n"
    prompt += "\nAnswer:"
    return prompt

def get_model_prediction(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
            temperature=1.0
        )
    generated_text = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    )
    answer = generated_text.strip()[:1].upper()
    if answer not in ["A", "B", "C", "D"]:
        for char in generated_text.upper():
            if char in ["A", "B", "C", "D"]:
                answer = char
                break
        else:
            answer = "A"
    return answer

def evaluate_subject(model, tokenizer, subject):
    print(f"Evaluating: {subject}")
    dataset = load_dataset("cais/mmlu", subject, split="test")
    correct = 0
    total = 0

    for example in tqdm(dataset, desc=f"Testing {subject}"):
        question = example["question"]
        choices = example["choices"]
        correct_answer_idx = example["answer"]
        correct_answer = ["A", "B", "C", "D"][correct_answer_idx]

        prompt = format_mmlu_prompt(question, choices)
        predicted_answer = get_model_prediction(model, tokenizer, prompt)

        if predicted_answer == correct_answer:
            correct += 1
        total += 1

    accuracy = (correct / total * 100) if total > 0 else 0
    print(f"Result: {correct}/{total} = {accuracy:.2f}%")
    return {"subject": subject, "correct": correct, "total": total, "accuracy": accuracy}

def main():
    print("="*70)
    print("ASTRONOMY Evaluation")
    print("="*70)

    model, tokenizer = load_model_and_tokenizer()
    start_time = datetime.now()

    results = []
    for subject in MMLU_SUBJECTS:
        result = evaluate_subject(model, tokenizer, subject)
        results.append(result)

    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()

    output_file = f"astronomy_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(output_file, "w") as f:
        json.dump({"results": results, "duration_seconds": duration}, f, indent=2)

    print(f"\nCompleted in {duration:.1f} seconds")
    print(f"Results saved to: {output_file}")

if __name__ == "__main__":
    main()


Writing llama_mmlu_astronomy.py


In [ ]:
%%writefile llama_mmlu_business.py
"""
Llama 3.2-1B MMLU Evaluation Script - BUSINESS ETHICS ONLY
"""

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import json
from tqdm.auto import tqdm
from datetime import datetime
import sys

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"
USE_GPU = True
MAX_NEW_TOKENS = 1
QUANTIZATION_BITS = 4
MMLU_SUBJECTS = ["business_ethics"]  # ONLY BUSINESS ETHICS

def get_quantization_config():
    if QUANTIZATION_BITS == 4:
        return BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )
    return None

def load_model_and_tokenizer():
    print(f"Loading {MODEL_NAME} for BUSINESS ETHICS evaluation...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    quant_config = get_quantization_config()

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=quant_config,
        device_map="auto",
        low_cpu_mem_usage=True
    )
    model.eval()
    return model, tokenizer

def format_mmlu_prompt(question, choices):
    choice_labels = ["A", "B", "C", "D"]
    prompt = f"{question}\n\n"
    for label, choice in zip(choice_labels, choices):
        prompt += f"{label}. {choice}\n"
    prompt += "\nAnswer:"
    return prompt

def get_model_prediction(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False,
            temperature=1.0
        )
    generated_text = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    )
    answer = generated_text.strip()[:1].upper()
    if answer not in ["A", "B", "C", "D"]:
        for char in generated_text.upper():
            if char in ["A", "B", "C", "D"]:
                answer = char
                break
        else:
            answer = "A"
    return answer

def evaluate_subject(model, tokenizer, subject):
    print(f"Evaluating: {subject}")
    dataset = load_dataset("cais/mmlu", subject, split="test")
    correct = 0
    total = 0

    for example in tqdm(dataset, desc=f"Testing {subject}"):
        question = example["question"]
        choices = example["choices"]
        correct_answer_idx = example["answer"]
        correct_answer = ["A", "B", "C", "D"][correct_answer_idx]

        prompt = format_mmlu_prompt(question, choices)
        predicted_answer = get_model_prediction(model, tokenizer, prompt)

        if predicted_answer == correct_answer:
            correct += 1
        total += 1

    accuracy = (correct / total * 100) if total > 0 else 0
    print(f"Result: {correct}/{total} = {accuracy:.2f}%")
    return {"subject": subject, "correct": correct, "total": total, "accuracy": accuracy}

def main():
    print("="*70)
    print("BUSINESS ETHICS Evaluation")
    print("="*70)

    model, tokenizer = load_model_and_tokenizer()
    start_time = datetime.now()

    results = []
    for subject in MMLU_SUBJECTS:
        result = evaluate_subject(model, tokenizer, subject)
        results.append(result)

    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()

    output_file = f"business_ethics_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(output_file, "w") as f:
        json.dump({"results": results, "duration_seconds": duration}, f, indent=2)

    print(f"\nCompleted in {duration:.1f} seconds")
    print(f"Results saved to: {output_file}")

if __name__ == "__main__":
    main()


Writing llama_mmlu_business.py


In [ ]:
%%bash
time { python llama_mmlu_astronomy.py ; python llama_mmlu_business.py ; }

ASTRONOMY Evaluation
Loading meta-llama/Llama-3.2-1B-Instruct for ASTRONOMY evaluation...
Evaluating: astronomy
Result: 72/152 = 47.37%

Completed in 13.4 seconds
Results saved to: astronomy_results_20260222_181400.json
BUSINESS ETHICS Evaluation
Loading meta-llama/Llama-3.2-1B-Instruct for BUSINESS ETHICS evaluation...
Evaluating: business_ethics
Result: 39/100 = 39.00%

Completed in 10.6 seconds
Results saved to: business_ethics_results_20260222_181430.json


Testing business_ethics: 100%|██████████| 100/100 [00:07<00:00, 13.11it/s]

real	1m9.236s
user	0m48.995s
sys	0m5.109s


Without Ollama Server - Parallel Execution

In [ ]:
%%bash
time { python llama_mmlu_astronomy.py & python llama_mmlu_business.py & wait; }

BUSINESS ETHICS Evaluation
Loading meta-llama/Llama-3.2-1B-Instruct for BUSINESS ETHICS evaluation...
ASTRONOMY Evaluation
Loading meta-llama/Llama-3.2-1B-Instruct for ASTRONOMY evaluation...
Evaluating: business_ethics
Evaluating: astronomy
Result: 39/100 = 39.00%

Completed in 17.9 seconds
Results saved to: business_ethics_results_20260222_181918.json
Result: 72/152 = 47.37%

Completed in 23.3 seconds
Results saved to: astronomy_results_20260222_181923.json


Testing astronomy: 100%|██████████| 152/152 [00:21<00:00,  7.05it/s]

real	0m48.855s
user	1m10.995s
sys	0m5.926s


With Ollama Server - Sequential Execution

In [ ]:
%%writefile llama_mmlu_astronomy_ollama.py
"""
Llama 3.2-1B MMLU Evaluation Script - ASTRONOMY (Ollama)
"""

import requests
from datasets import load_dataset
import json
from tqdm.auto import tqdm
from datetime import datetime

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3.2:1b"
MMLU_SUBJECTS = ["astronomy"]

def format_mmlu_prompt(question, choices):
    choice_labels = ["A", "B", "C", "D"]
    prompt = f"{question}\n\n"
    for label, choice in zip(choice_labels, choices):
        prompt += f"{label}. {choice}\n"
    prompt += "\nAnswer:"
    return prompt

def get_model_prediction(prompt):
    try:
        response = requests.post(
            OLLAMA_URL,
            json={
                "model": MODEL_NAME,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0,
                    "num_predict": 1
                }
            },
            timeout=30
        )
        result = response.json()
        answer = result['response'].strip()[:1].upper()

        if answer not in ["A", "B", "C", "D"]:
            for char in result['response'].upper():
                if char in ["A", "B", "C", "D"]:
                    answer = char
                    break
            else:
                answer = "A"
        return answer
    except Exception as e:
        print(f"Error: {e}")
        return "A"

def evaluate_subject(subject):
    print(f"Evaluating: {subject}")
    dataset = load_dataset("cais/mmlu", subject, split="test")
    correct = 0
    total = 0

    for example in tqdm(dataset, desc=f"Testing {subject}"):
        question = example["question"]
        choices = example["choices"]
        correct_answer_idx = example["answer"]
        correct_answer = ["A", "B", "C", "D"][correct_answer_idx]

        prompt = format_mmlu_prompt(question, choices)
        predicted_answer = get_model_prediction(prompt)

        if predicted_answer == correct_answer:
            correct += 1
        total += 1

    accuracy = (correct / total * 100) if total > 0 else 0
    print(f"Result: {correct}/{total} = {accuracy:.2f}%")
    return {"subject": subject, "correct": correct, "total": total, "accuracy": accuracy}

def main():
    print("="*70)
    print("ASTRONOMY Evaluation (Ollama)")
    print("="*70)

    start_time = datetime.now()
    results = []

    for subject in MMLU_SUBJECTS:
        result = evaluate_subject(subject)
        results.append(result)

    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()

    output_file = f"astronomy_ollama_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(output_file, "w") as f:
        json.dump({"results": results, "duration_seconds": duration}, f, indent=2)

    print(f"\nCompleted in {duration:.1f} seconds")
    print(f"Results saved to: {output_file}")

if __name__ == "__main__":
    main()


Writing llama_mmlu_astronomy_ollama.py


In [ ]:
%%writefile llama_mmlu_business_ollama.py
"""
Llama 3.2-1B MMLU Evaluation Script - BUSINESS ETHICS (Ollama)
"""

import requests
from datasets import load_dataset
import json
from tqdm.auto import tqdm
from datetime import datetime

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3.2:1b"
MMLU_SUBJECTS = ["business_ethics"]

def format_mmlu_prompt(question, choices):
    choice_labels = ["A", "B", "C", "D"]
    prompt = f"{question}\n\n"
    for label, choice in zip(choice_labels, choices):
        prompt += f"{label}. {choice}\n"
    prompt += "\nAnswer:"
    return prompt

def get_model_prediction(prompt):
    try:
        response = requests.post(
            OLLAMA_URL,
            json={
                "model": MODEL_NAME,
                "prompt": prompt,
                "stream": False,
                "options": {
                    "temperature": 0,
                    "num_predict": 1
                }
            },
            timeout=30
        )
        result = response.json()
        answer = result['response'].strip()[:1].upper()

        if answer not in ["A", "B", "C", "D"]:
            for char in result['response'].upper():
                if char in ["A", "B", "C", "D"]:
                    answer = char
                    break
            else:
                answer = "A"
        return answer
    except Exception as e:
        print(f"Error: {e}")
        return "A"

def evaluate_subject(subject):
    print(f"Evaluating: {subject}")
    dataset = load_dataset("cais/mmlu", subject, split="test")
    correct = 0
    total = 0

    for example in tqdm(dataset, desc=f"Testing {subject}"):
        question = example["question"]
        choices = example["choices"]
        correct_answer_idx = example["answer"]
        correct_answer = ["A", "B", "C", "D"][correct_answer_idx]

        prompt = format_mmlu_prompt(question, choices)
        predicted_answer = get_model_prediction(prompt)

        if predicted_answer == correct_answer:
            correct += 1
        total += 1

    accuracy = (correct / total * 100) if total > 0 else 0
    print(f"Result: {correct}/{total} = {accuracy:.2f}%")
    return {"subject": subject, "correct": correct, "total": total, "accuracy": accuracy}

def main():
    print("="*70)
    print("BUSINESS ETHICS Evaluation (Ollama)")
    print("="*70)

    start_time = datetime.now()
    results = []

    for subject in MMLU_SUBJECTS:
        result = evaluate_subject(subject)
        results.append(result)

    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()

    output_file = f"business_ethics_ollama_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(output_file, "w") as f:
        json.dump({"results": results, "duration_seconds": duration}, f, indent=2)

    print(f"\nCompleted in {duration:.1f} seconds")
    print(f"Results saved to: {output_file}")

if __name__ == "__main__":
    main()


Writing llama_mmlu_business_ollama.py


In [ ]:
%%bash
time { python llama_mmlu_astronomy_ollama.py ; python llama_mmlu_business_ollama.py ; }

ASTRONOMY Evaluation (Ollama)
Evaluating: astronomy
Result: 27/152 = 17.76%

Completed in 52.7 seconds
Results saved to: astronomy_ollama_results_20260222_184502.json
BUSINESS ETHICS Evaluation (Ollama)
Evaluating: business_ethics
Result: 30/100 = 30.00%

Completed in 32.9 seconds
Results saved to: business_ethics_ollama_results_20260222_184536.json


Testing business_ethics: 100%|██████████| 100/100 [00:31<00:00,  3.20it/s]

real	1m28.845s
user	0m4.107s
sys	0m0.446s


With Ollama Server - Parallel Execution

In [ ]:
%%bash
time { python llama_mmlu_astronomy_ollama.py & python llama_mmlu_business_ollama.py & wait; }

BUSINESS ETHICS Evaluation (Ollama)
Evaluating: business_ethics
ASTRONOMY Evaluation (Ollama)
Evaluating: astronomy
Result: 30/100 = 30.00%

Completed in 62.5 seconds
Results saved to: business_ethics_ollama_results_20260222_184646.json
Result: 27/152 = 17.76%

Completed in 80.5 seconds
Results saved to: astronomy_ollama_results_20260222_184704.json


Testing astronomy: 100%|██████████| 152/152 [01:18<00:00,  1.93it/s]

real	1m22.489s
user	0m5.087s
sys	0m0.539s
